In [1]:
!which pip

/home/gcinizwe/miniconda3/envs/venv_dpd_py38/bin/pip


In [2]:
!which python

/home/gcinizwe/miniconda3/envs/venv_dpd_py38/bin/python


In [3]:
!ls

Untitled.ipynb	android2	    intersection.py  project_titles.pkl
android.zip	filtered_repos.zip  new_directory    train_data


In [ ]:
import os
import pickle
import fnmatch
import shutil


def delete_dir(path):
    shutil.rmtree(path)


def read_pickle_file(file_path):
    with open(file_path, 'rb') as f:
        # Load the serialized object from the file
        obj = pickle.load(f)
    return obj


folder_titles = read_pickle_file("project_titles.pkl")

# Total items: 919
stripped_titles = [os.path.normpath(t).split(os.sep)[-1] for t in folder_titles]

assert len(stripped_titles) == 919


def find_duplicates():
    duplicates = 0

    for dirname in os.listdir("./android2"):

        if dirname in stripped_titles:
            # Uncomment the following code to delete the files
            # full_path=folder_titles[stripped_titles.index(dirname)]
            # delete_dir(full_path)
            duplicates += 1

    print("dupl: ", duplicates)


# for root, dirnames_parent, filenames in os.walk('./new_directory'):
#     for dirname in dirnames_parent:
#         dir_path = './new_directory/' + f'{dirname}'
#         src_path = find_src_path(dir_path)
#         get_java_paths()
        # for _, dirnames, filenames in os.walk(dir_path):
        #     breakpoint()
        #     if not fnmatch.fnmatch(filename, '*.java'):
        #         file_path = os.path.join(root, filename)
        #         breakpoint()
        #         os.remove(file_path)



In [3]:
from pathlib import Path
import pandas as pd
import os

In [4]:

def find_src_path(android_dir):
    for root, dirs, files in os.walk(android_dir):
        if (root.endswith('src')):
            return root


def find_files_recursively(src_dir: str):
    """find all .java files recursively from src dir"""
    java_files = []
    for root, dirs, files in os.walk(src_dir):
        for file in files:
            if file.endswith('.java'):
                java_files.append(root + '/' + file)
    return java_files




def get_java_paths(base) -> pd.DataFrame:
    all_dirs = []
    all_java_paths = []
    for android_dir in os.listdir(base):
        src = find_src_path(base / android_dir)
        if src:
            java_paths = find_files_recursively(src)
            all_dirs.extend(len(java_paths) * [android_dir])
            all_java_paths.extend(java_paths)
        else:
            print("SRC not FOUND")
    df = pd.DataFrame({"projects": all_dirs, "java_paths": all_java_paths})
    return df



df_all = get_java_paths(Path('./new_directory'))
breakpoint()

SRC not FOUND
SRC not FOUND
SRC not FOUND


In [5]:
df_all.shape

(14400, 2)

In [6]:
df_all

,projects,java_paths
0,adblockplusandroid,new_directory/adblockplusandroid/src/org/liter...
1,adblockplusandroid,new_directory/adblockplusandroid/src/org/apach...
2,adblockplusandroid,new_directory/adblockplusandroid/src/org/apach...
3,adblockplusandroid,new_directory/adblockplusandroid/src/org/apach...
4,adblockplusandroid,new_directory/adblockplusandroid/src/org/adblo...
...,...,...
14395,ArcLayout,new_directory/ArcLayout/library/src/main/java/...
14396,ArcLayout,new_directory/ArcLayout/library/src/main/java/...
14397,ArcLayout,new_directory/ArcLayout/library/src/main/java/...
14398,ArcLayout,new_directory/ArcLayout/library/src/main/java/...


In [7]:
def read_code(path):
    with open(path, 'r', encoding='cp1252', errors='ignore') as p:
        return p.read()

In [8]:
all_code = []

for _, (project, path) in df_all.iterrows():
    all_code.append(read_code(path))


In [31]:
# all_code[0]

In [9]:
from tqdm import tqdm

t_path = Path('./android2')

all_code_t = []
empty_android_projects = 0


for i, android_dir in tqdm(enumerate(os.listdir(t_path))):
    data_dir = t_path / android_dir
    java_files = os.listdir(data_dir)
    if not java_files:
        empty_android_projects += 1
    else:
        for jfile in java_files:
            jfull_path = data_dir / jfile
            all_code_t.append(read_code(jfull_path))

1040it [00:29, 34.84it/s]


In [10]:
"The number of projects that are empty: %d" % empty_android_projects

'The number of projects that are empty: 186'

In [11]:
len(all_code), len(all_code_t)

(14400, 140653)

In [31]:
all_data_java = all_code + all_code_t
len(all_data_java)

155053

In [32]:
import random

# Set the random seed to ensure the same random samples every time
random.seed(42)

# Calculate 50% of the data size
sample_size = int(0.5 * len(all_data_java))

# Sample 10% of the data
random_sample_java = random.sample(all_data_java, sample_size)

print(len(random_sample_java))
all_data_java = random_sample_java

77526


In [33]:
def write_lines(path, data):
    with open(path, 'a') as f:
        for code in data:  # all_code[:len(all_code) - percent_10]:
            f.write(code.replace('\n', '\\n'))
            f.write('\n')

In [34]:
len(all_data_java)

77526

In [35]:
percent_10 = int(len(all_data_java) * 0.1)
percent_10 < len(all_data_java)

True

In [36]:
len(all_data_java[-percent_10:])

7752

In [37]:
!ls

'Classification Head - BigBird.ipynb'   coach_repos_zip.zip
 LM_Tuning_BigBird.ipynb	        coach_repos_zip.zipranvcygstmp
 __MACOSX			        filtered_repos.zip
 android.zip			        intersection.py
 android2			        logs_output
 artifacts			        model_output
 bigbird_tuning.py		        new_directory
 catboost_info			        project_titles.pkl
 ck_results_of_repos.csv	        train_data
 coach_repos_zip		        train_data_sample_50percent


In [38]:
# !rm -f train_data/*

In [39]:
# !rm -f train_data_sample_50percent/*

In [40]:
write_lines("train_data_sample_50percent/train_data.txt", all_data_java[:len(all_data_java) - percent_10])

In [41]:
write_lines("train_data_sample_50percent/test_data.txt", all_data_java[-percent_10:])

### Training Code

In [47]:
import transformers
transformers.__version__

'4.24.0'

In [48]:
transformers.__version__

'4.24.0'

In [49]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [50]:
from transformers import AutoTokenizer, BigBirdModel
import torch

MODEL_NAME = 'google/bigbird-roberta-base'

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [51]:
def count_tokens_len(code_input):
    inputs = tokenizer(code_input , return_tensors="pt")
    return inputs.input_ids.shape

In [53]:
# from collections import defaultdict
# project_wise_counts = defaultdict(list)

# for _, (project, path) in df_all.iterrows():
#     count = count_tokens_len(read_code(path))
#     project_wise_counts[project].append(count[1])
!ls

'Classification Head - BigBird.ipynb'   coach_repos_zip.zip
 LM_Tuning_BigBird.ipynb	        coach_repos_zip.zipranvcygstmp
 __MACOSX			        filtered_repos.zip
 android.zip			        intersection.py
 android2			        logs_output
 artifacts			        model_output
 bigbird_tuning.py		        new_directory
 catboost_info			        project_titles.pkl
 ck_results_of_repos.csv	        train_data
 coach_repos_zip		        train_data_sample_50percent


In [54]:
import torch
from transformers import BigBirdForMaskedLM, BigBirdTokenizer, BigBirdConfig, DataCollatorForLanguageModeling
from transformers import LineByLineTextDataset, Trainer, TrainingArguments

# Load pre-trained model
model_name = "google/bigbird-roberta-base"
config = BigBirdConfig.from_pretrained(MODEL_NAME)

model = BigBirdForMaskedLM.from_pretrained(MODEL_NAME).to(device)

# Load dataset
train_file = "train_data_sample_50percent/train_data.txt"
train_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path=train_file,
    block_size=config.block_size,
)

Some weights of the model checkpoint at google/bigbird-roberta-base were not used when initializing BigBirdForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BigBirdForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/home/gcinizwe/miniconda3/envs/venv_dpd_py38/lib/python3.8/site-packages/transformers/data/datasets/language_modeling.py:121: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transf

In [55]:
eval_file = "train_data_sample_50percent/test_data.txt"
eval_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path=eval_file,
    block_size=config.block_size,
)

In [56]:
# Prepare data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [58]:
!mkdir model_output_sample
!mkdir logs_output_sample

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
mkdir: cannot create directory ‘model_output_sample’: File exists
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
mkdir: cannot create directory ‘logs_output_sample’: File exists


In [60]:
# Set up training arguments
output_dir = "./model_output_sample" 
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=8,
    save_steps=1_000,
    save_total_limit=5,
    logging_dir="./logs_output_sample",
    logging_steps=1_000,
    learning_rate=5e-6,
    weight_decay=0.01,
    evaluation_strategy="steps",
    load_best_model_at_end=True,
)

# Create the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)


using `logging_steps` to initialize `eval_steps` to 1000
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [61]:
# Start training
trainer.train()

/home/gcinizwe/miniconda3/envs/venv_dpd_py38/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 69799
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 43625
  Number of trainable parameters = 128111286
You're using a BigBirdTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Attention type 'block_sparse' is not possible if sequence_length: 64 <= num global tokens: 2 * config.block_size + min. num sliding t

Step,Training Loss,Validation Loss
1000,1.930600,1.300754
2000,1.227600,1.012144


***** Running Evaluation *****
  Num examples = 7752
  Batch size = 8
Saving model checkpoint to ./model_output_sample/checkpoint-1000
Configuration saved in ./model_output_sample/checkpoint-1000/config.json
Model weights saved in ./model_output_sample/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 7752
  Batch size = 8
Saving model checkpoint to ./model_output_sample/checkpoint-2000
Configuration saved in ./model_output_sample/checkpoint-2000/config.json
Model weights saved in ./model_output_sample/checkpoint-2000/pytorch_model.bin


KeyboardInterrupt: 

In [29]:
!ls

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
LM_Tuning_BigBird.ipynb  filtered_repos.zip  model_output	 train_data
android.zip		 intersection.py     new_directory
android2		 logs_output	     project_titles.pkl
